In [ ]:
!pip install praat-parselmouth

In [ ]:
!pip install pyaudio

In [ ]:
!pip install scipy

In [ ]:
!pip install IPython

In [ ]:
import os
import numpy as np
from scipy.io import wavfile
import parselmouth 
from parselmouth.praat import call
from IPython.display import Audio
import pyaudio
import wave
import time
import pandas as pd
import csv

def record():
    chunk = 1024  # Record in chunks of 1024 samples
    sample_format = pyaudio.paInt16  # 16 bits per sample
    channels = 1
    fs = 22050  # Record at 44100 samples per second
    seconds = .25
    filename = "output.wav"

    p = pyaudio.PyAudio()  # Create an interface to PortAudio

    stream = p.open(format=sample_format,
                    channels=channels,
                    rate=fs,
                    frames_per_buffer=chunk,
                    input=True)

    frames = []  # Initialize array to store frames

# Store data in chunks for 3 seconds
    for i in range(0, int(fs / chunk * seconds)):
        data = stream.read(chunk)
        frames.append(data)

# Stop and close the stream 
    stream.stop_stream()
    stream.close()
# Terminate the PortAudio interface
    p.terminate()

# Save the recorded data as a WAV file
    wf = wave.open(filename, 'wb')
    wf.setnchannels(channels)
    wf.setsampwidth(p.get_sample_size(sample_format))
    wf.setframerate(fs)
    wf.writeframes(b''.join(frames))
    wf.close()
    
    fs, x = wavfile.read("output.wav")
    Audio(x,rate=fs)
    return fs, x

    
def formants_praat(x, fs):
        f0min, f0max  = 75, 300
        sound = parselmouth.Sound(x, sampling_frequency=fs) # read the sound
        pitch = sound.to_pitch()
        f0 = pitch.selected_array['frequency']
        formants = sound.to_formant_burg(time_step=0.010, maximum_formant=5000)
        
        f1_list, f2_list = [], []
        for t in formants.ts():
            f1 = formants.get_value_at_time(1, t)
            f2 = formants.get_value_at_time(2, t)
            if np.isnan(f1): f1 = 0
            if np.isnan(f2): f2 = 0
            f1_list.append(f1)
            f2_list.append(f2)
            
        return int(np.mean(f0)), int(np.mean(f1_list)), int(np.mean(f2_list))

vowels = ['i', 'ɪ', 'e', 'ɛ', 'æ', 'a', 'ɑ', 'ə', 'ɔ', 'o', 'ʊ', 'u']


name = input("Enter your name here: ")
date = input("Enter today's date as follows, using the underscore as indicated (MONTH NUMBER)_(DAY): ")
print(f"Hello, {name}! You will be asked to read a series of 12 words in order to determine your baseline values for formants 1 and 2.")
print("In the case of the dipthongs [eɪ], [aɪ] and [oʊ] in the words 'BAY', 'BUY' and 'BOW', sustain the first vowel, not the second.")
print()
go_ahead = input("Enter Y to continue:")
if go_ahead == "Y" or go_ahead == "y":
    secs = 2
    print("Get ready!")
    while secs > 0:
        time.sleep(1)  # Delay for 1 second
        secs -= 1
print()

word_list = ["BEE", "BIT", "BAY", "BED", "BAD", "BUY", "POD", "BUT", "PAW", "BOAT", "PUT", "BOO"]

time.sleep(1)
y = 0
results = []
while y < len(word_list):
    print(f"{y+1}.")
    print("Begin recording this word as soon as you see 'Go!': ", word_list[y])
    print("Sustain the vowel as steadily as possible while counting to 10.")
    go_ahead = input("Enter Y to continue:")
    if go_ahead == "Y" or go_ahead == "y":
        secs = 5
        print(f"Get ready: ")
        while secs > 0:
            print(f"{secs} seconds")
            time.sleep(1)
            secs -= 1
    print("GO!")
    time.sleep(1)
      
    fs, x = record()
    f0, f1, f2 = formants_praat(x, fs)
    f0_values = []
    f1_values = []
    f2_values = []
    while f0 > 0:
        f0_values.append(f0)
        f1_values.append(f1)
        f2_values.append(f2)
        fs, z = record()
        f0, f1, f2 = formants_praat(z, fs)
    f0_values.pop(0)
    f0_values.pop(-1)
    f1_values.pop(0)
    f1_values.pop(-1)
    f2_values.pop(0)
    f2_values.pop(-1)
    results.append([vowels[y], int(np.mean(f0_values)), int(np.mean(f1_values)), int(np.mean(f2_values))])
    print()  
    y += 1
    
filename = f"{name} {date}.csv"
with open(filename, 'w', newline='') as file:
    writer = csv.writer(file)
    field = ["NAME", "DATE", "VOWEL", "AVG F0", "AVG F1", "AVG F2"]
    writer.writerow(field)
    for result in results:
        writer.writerow([name, date, result[0], result[1], result[2], result[3]])    

In [ ]:
#RUN THIS IF YOU WANT TO SEE YOUR RESULTS!

all_data = []
for result in results:
    
    data = {}
    data["NAME"] = name
    data["DATE"] = date
    data["VOWEL"] = result[0]
    data["AVG F0"] = result[1]
    data["AVG F1"] = result[2]
    data["AVG F2"] = result[3]
    all_data.append(data)
    
df =pd.DataFrame(all_data)
df
